In [1]:
# Package imports
import numpy as np
import copy
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
import pandas as pd
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [90]:
data = pd.read_csv("data_10000.csv")
print("data type:",type(data))
print("data shape:", data.shape)
print("data:",data)

# Split the data into train and test
train_dataset = data.sample(frac=0.8, random_state=0)
test_dataset = data.drop(train_dataset.index)

# Normalize (divide positions (x1,x3,x5,0-5) by 5 and values (x2,x4,x6,0-100) by 100)
x_train = train_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,0:6]
y_train = train_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,6:12]
x_test = test_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,0:6]
y_test = test_dataset.multiply([1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100,1/5,1/100]).iloc[:,6:12]
print(data.shape, train_dataset.shape, test_dataset.shape)
print(x_train.shape)

data type: <class 'pandas.core.frame.DataFrame'>
data shape: (10000, 12)
data:       x1  x2  x3  x4  x5  x6  y1  y2  y3  y4  y5  y6
0      1   1   2   2   3   3   0   2   4   3   5   6
1      1   1   2   3   3   2   0   1   4   4   5   6
2      1   1   2   6   3   3   0   2   4   7   5  10
3      0   1   1   5   2   2   3   6   4   7   5  13
4      0   2   1   1   4   7   2   6   3   3   5  10
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
9995   1  23   4  43   5  90   0  24   2  20   3  47
9996   0  23   4  48   5  95   1  24   2  24   3  47
9997   0  23   4  43   5  90   1  24   2  19   3  47
9998   1  24   3  48   5  95   0  24   2  23   4  47
9999   0  24   3  48   5  91   1  24   2  19   4  43

[10000 rows x 12 columns]
(10000, 12) (8000, 12) (2000, 12)
(8000, 6)


In [91]:
# Definde Sequential Model
model = keras.Sequential(
    [   keras.Input(shape=(6,)),
        keras.layers.Dense(48, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(192, activation='relu'),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.Dense(6)
    ]
)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 48)                336       
                                                                 
 dense_48 (Dense)            (None, 96)                4704      
                                                                 
 dense_49 (Dense)            (None, 192)               18624     
                                                                 
 dense_50 (Dense)            (None, 96)                18528     
                                                                 
 dense_51 (Dense)            (None, 6)                 582       
                                                                 
Total params: 42774 (167.09 KB)
Trainable params: 42774 (167.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [92]:
# loss, optimizer and metrics
model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=['mae'])

In [105]:
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') <= 9.0e-08):
            print("\n\n\nReached 1.0e-07 loss value so stopping training!\n\n\n")
            self.model.stop_training = True
trainingStopCallback = haltCallback()

In [106]:
# training
model.fit(x_train, y_train, epochs=100,batch_size=64, verbose=1, callbacks=[trainingStopCallback])

Epoch 1/100
125/125 [==============================] - 1s 4ms/step - loss: 8.3032e-07 - mae: 6.3755e-04
Epoch 2/100
125/125 [==============================] - 1s 4ms/step - loss: 7.3225e-07 - mae: 6.3660e-04
Epoch 3/100
125/125 [==============================] - 0s 4ms/step - loss: 3.1113e-07 - mae: 4.2281e-04
Epoch 4/100
125/125 [==============================] - 1s 5ms/step - loss: 4.3753e-07 - mae: 4.4856e-04
Epoch 5/100
125/125 [==============================] - 1s 4ms/step - loss: 4.4161e-07 - mae: 4.8452e-04
Epoch 6/100
125/125 [==============================] - 1s 4ms/step - loss: 1.7854e-06 - mae: 9.5749e-04
Epoch 7/100
125/125 [==============================] - 1s 4ms/step - loss: 3.5519e-07 - mae: 4.3957e-04
Epoch 8/100
125/125 [==============================] - 1s 4ms/step - loss: 9.6535e-08 - mae: 2.3264e-04
Epoch 9/100
125/125 [==============================] - 1s 4ms/step - loss: 7.3364e-07 - mae: 5.8109e-04
Epoch 10/100
125/125 [==============================] - 0s 4ms/s

In [128]:
x1 = x_test.iloc[0:1,:]
y1 = y_test.iloc[0:1,:]
x1_pred = model.predict(x1, verbose=0)
print('x1',(x1*[5,100,5,100,5,100]).astype(int))
print('y1',(y1*[5,100,5,100,5,100]).astype(int))
print('prediction: ',np.round(x1_pred*[5,100,5,100,5,100]).astype(int))

#x2 = x_test.iloc[1:2,:]
#y2 = y_test.iloc[1:2,:]
#x2_pred = model.predict(x2, verbose=0)
#print((y2*[5,100,5,100,5,100]).astype(int))
#print(np.round(x2_pred*[5,100,5,100,5,100]).astype(int))


x1    x1  x2  x3  x4  x5  x6
0   1   1   2   2   3   3
y1    y1  y2  y3  y4  y5  y6
0   0   2   4   3   5   6
prediction:  [[0 2 4 3 5 6]]


In [109]:
pred = model.predict(x_test)
# De-Normalize
x_round = np.round(x_test*[5,100,5,100,5,100]).astype(int)
y_round = np.round(y_test*[5,100,5,100,5,100]).astype(int)
pred_round = np.round(pred*[5,100,5,100,5,100]).astype(int)
test_result = pred_round - y_round
test_result_row_sums = test_result.abs().sum(axis=1)
num_correct= len(test_result_row_sums[test_result_row_sums == 0])
print('Number of correct solutions:',num_correct,', out of ',test_result.shape[0], (100*num_correct)/test_result.shape[0], '%')

63/63 [==============================] - 0s 3ms/step
Number of correct solutions: 1987 , out of  2000 99.35 %


In [98]:
model.save("mathpyramid_1987_correct")

INFO:tensorflow:Assets written to: mathpyramid_1987_correct\assets


INFO:tensorflow:Assets written to: mathpyramid_1987_correct\assets


In [108]:
model = keras.models.load_model("mathpyramid_1987_correct")
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 48)                336       
                                                                 
 dense_48 (Dense)            (None, 96)                4704      
                                                                 
 dense_49 (Dense)            (None, 192)               18624     
                                                                 
 dense_50 (Dense)            (None, 96)                18528     
                                                                 
 dense_51 (Dense)            (None, 6)                 582       
                                                                 
Total params: 42774 (167.09 KB)
Trainable params: 42774 (167.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [135]:
# Predict with higher values (<= 1000) -> works too
data = {'x1': [0/5],
        'x2': [175/1000],
        'x3': [1/5],
        'x4': [114/1000],
        'x5': [2/5],
        'x6': [102/1000]
        }  
# Create the pandas DataFrame
df = pd.DataFrame(data)
print(df)
pred = model.predict(df)
print(np.round(pred*[5,1000,5,1000,5,1000]).astype(int))

    x1     x2   x3     x4   x5     x6
0  0.0  0.175  0.2  0.114  0.4  0.102
1/1 [==============================] - 0s 26ms/step
[[  3 289   4 216   5 505]]
